In [1]:
from elasticsearch import Elasticsearch
import pandas as pd
import numpy as np

import sys, os

### Get all trades

In [3]:
import joblib
with open('trades', 'rb') as file:
    trades = joblib.load('trades')

### Get min and max dates

In [4]:
dates = []
for trade in trades:
    dates.append(trade['_source']['initTime'])

In [5]:
dates = pd.to_datetime(dates)

In [6]:
dates.min(), dates.max()

(Timestamp('2019-07-02 00:15:00'), Timestamp('2019-07-17 21:05:00'))

In [7]:
date_range = pd.date_range(dates.min(), dates.max(), freq='5T')

### Portfolio dictionary

In [8]:
p_dict = {str(key) : {} for key in date_range}

### Get all trades

In [68]:
stats = []
for trade in trades:
    trade = trade['_source']
    if trade['executionLogic'] != 'NO FILL':
        r = (trade['direction'] * (trade['avgCostOnClose'] - trade['avgCost'])) / (0.2 * trade['tickIncrement'])
        stats.append([str(pd.to_datetime([trade['initTime']])[0]),
                      trade['ticker'], trade['direction'], len(trade['numUpdates']), trade['closingTime'], r])

In [69]:
df = pd.DataFrame(stats, columns = ['InitTime', 'Ticker', 'Direction', 'Length', 'ClosingTime', 'Realized']).sort_values('InitTime', ascending = True)

### Fill the Position Dict

In [70]:
keys = list(p_dict.keys())
for time, ticker, direction, length, ctime, r in df.values:
    time = time[:-2] + '00'
    idx = keys.index(time)
    active_keys = keys[idx:idx+length+1]
    for key in active_keys:
        p_dict[key][ticker] = {'dir' : direction, 'r' : r}

### Check for extra risk

In [78]:
for time in p_dict:
    
    positions = p_dict[time]
    
    if len(positions) == 0:
        continue
        
    keys, items = list(zip(*positions.items()))
    
    tickers = []
    [tickers.extend([ticker[:3], ticker[3:]]) for ticker in keys]
    tickers = pd.Series(tickers).value_counts()
    main = tickers[tickers > 1].index.values.tolist()
    
    risk = {}
    
    for ticker in main:
        
        numers = []
        denoms = []

        for key in keys:
            if ticker in key[:3]:
                numers.append(key)
            elif ticker in key[3:]:
                denoms.append(key)

        denom_directions = []
        numer_directions = []
        realized = []
        for denom in denoms:
            denom_directions.append(p_dict[time][denom]['dir'])
            realized.append(p_dict[time][denom]['r'])
        for numer in numers:
            numer_directions.append(p_dict[time][numer]['dir'])
            realized.append(p_dict[time][numer]['r'])
        numer_direction = (np.array(numer_directions) * -1).tolist()
        risk[ticker] = sum(numer_directions + denom_directions)
    
    p_dict[time]['risk'] = {'risk' : risk, 'realized' : sum(realized)}

In [82]:
for key, value in p_dict.items():
    if 'risk' in value:
        print(key, value['risk'])

2019-07-02 00:15:00 {'risk': {'JPY': 3}, 'realized': 193.9999999999884}
2019-07-02 00:20:00 {'risk': {'JPY': 3}, 'realized': 193.9999999999884}
2019-07-02 00:25:00 {'risk': {'JPY': 3}, 'realized': 193.9999999999884}
2019-07-02 00:30:00 {'risk': {'JPY': 3}, 'realized': 193.9999999999884}
2019-07-02 00:35:00 {'risk': {'JPY': 3, 'EUR': 0}, 'realized': -10.999999999985008}
2019-07-02 00:40:00 {'risk': {'EUR': 0, 'JPY': 2}, 'realized': 146.0000000000008}
2019-07-02 00:45:00 {'risk': {}, 'realized': 146.0000000000008}
2019-07-02 00:50:00 {'risk': {}, 'realized': 146.0000000000008}
2019-07-02 00:55:00 {'risk': {}, 'realized': 146.0000000000008}
2019-07-02 01:00:00 {'risk': {}, 'realized': 146.0000000000008}
2019-07-02 01:05:00 {'risk': {}, 'realized': 146.0000000000008}
2019-07-02 01:10:00 {'risk': {}, 'realized': 146.0000000000008}
2019-07-02 01:15:00 {'risk': {}, 'realized': 146.0000000000008}
2019-07-02 03:45:00 {'risk': {}, 'realized': 146.0000000000008}
2019-07-02 03:50:00 {'risk': {'JPY